# ISMI2018 Lymphocyte Detection Project
This is the main notebook for the final ISMI2018 project by Group 6.


*Detection of lymphocytes in histopathology whole-slide images of breast, colon and prostate cancer, stained with immunohistochemistry*

### Team 6
Our group consists out of four people:
* **Brian Westerweel (Data Science)** - `B.Westerweel@student.ru.nl`
* **Christoph Schmidl (Data Science)** - `c.schmidl@student.ru.nl`
* **Gijs van der Meijde (Software Science)** - `G.vanderMeijde@student.ru.nl`
* **Jeffrey Luppus (Data Science)** - `J.Luppes@student.ru.nl`

*Supervisor: Francesco Ciompi*

Group Github page: https://github.com/jeffluppes/ISMI2018 *feel free to work in branches.*

* ```x``` = unavailable this date (e.g. unable to work on anything)
* ```-``` = reduced availability (e.g. you might be able to skype or work a couple of hours)

| Name  | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | *Notes* |
|-------|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|---------|
| Jeff  | - | - | x | x | x |  - |  - |  - |  - |    |  - |    |    |    |    |    |    |    |  x |    |    | *2 hrs a day for pet projects during work days, internship ends 15th* |
| Gijs  | x | x | x | x | x |  x |  x |  x |  x |  x |  - |    |    |  - |  - |    |  - |    |    |    |    | *has short term deadline until the 14th*        |
| Brian |   |   |   | - | - |  x |  x |  - |  - |  - |  - |  x |  x |  - |  - |  - |  - | x  |  x |  x |    |         |
| Chris |   |   |   |   |   |    |  x |  x |  x |  x |    |    |    |    |  x |  x |  x |    |    |    |    |         | 


## Goals for this week (June 11 to June 18)

**MAIN GOALS**
* Use all of the training and validation data that we have available
* ~~Implement a function to return the coordinates of a mask (centroid calculation)~~
* Saving predictions (images) to disk
* ~~Make a submission file~~
* Use generator and make sensible augmentations
* Make annotations with open source tools 
* Make our first submission

**Other goals:**

* Implement loss plotting function
* Implement early stopping
* Implement a custom loss function
* Run predictions on the entire validation set

**Backlog and minor TODOs**
* Implement image warping
* Get data augmentation working
* Implement a function to count lymphocytes

Be aware of the following:

* **Next contact moment**: Monday at 10 skype call > New moment to be scheduled
* **Next group meeting**: Friday and Monday


## Important Deadlines

* ~~**Mid-term presentation:** June 4, 9:00 - 12:00, room 616 Huygen building~~
* **Deadline project:** 25th of june
* **Final presentation:** July 2, 9:00 - 12:00, room 616 Huygen building


## Resources
The two papers Francesco sent:

* https://openreview.net/pdf?id=rk0xLisiM
* https://openreview.net/pdf?id=S10IfW2oz

Kaggle Contest about cell detection (Data Science Bowl 2018)

* https://www.kaggle.com/c/data-science-bowl-2018

Particular Kernels of Interest (KoIs)

* https://www.kaggle.com/weiji14/yet-another-keras-u-net-data-augmentation
* https://www.kaggle.com/piotrczapla/tensorflow-u-net-starter-lb-0-34
* https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277?scriptVersionId=2164855

Other resources

* https://spark-in.me/post/unet-adventures-part-one-getting-acquainted-with-unet
* https://towardsdatascience.com/medical-image-segmentation-part-1-unet-convolutional-networks-with-interactive-code-70f0f17f46c6 (Christoph)
* https://colab.research.google.com/drive/1BgCDxVdVc0MAe_kC0waMGUV9ShcWW0hM (Christoph)
* https://keunwoochoi.wordpress.com/2017/10/11/u-net-on-keras-2-0/ (Jeff)
* https://github.com/jocicmarko/ultrasound-nerve-segmentation (Jeff)
* https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/ (Gijs)

### MOOCs if you have spare time to waste (I havent)
http://course.fast.ai/part2.html
http://course.fast.ai/lessons/lesson14.html

In [ ]:
# System
from os.path import join, basename, dirname, exists  
import os  
from glob import glob
import csv
import random 
from tqdm import tqdm
from PIL import Image
Image.MAX_IMAGE_PIXELS = 2201136600 #because Pillow will whine to no end if you feed it big images. You're welcome.
import math

# Computational
import pandas as pd  
import numpy as np  
from matplotlib import pyplot as plt  

# Keras
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Input, Model
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Flatten, Dropout, UpSampling2D, core
from keras.callbacks import ModelCheckpoint


# Other
import seaborn as sns
import skimage
import sklearn
from scipy.ndimage import imread  
from IPython.display import clear_output
from sklearn.metrics import precision_recall_curve

## 0.1 Tracking time
This tracks the computation time for the entire notebook. See results in the last cell.


In [ ]:
import time
start_time = time.time()

In [ ]:
#user = 'ruc0028' # Jeffrey = True
user = 'ruc0030' # Gijs = True

## Cartesius file paths

In [ ]:
train_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/train_images'
validation_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/validation_images'
test_dir = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/test_images'
train_points = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/training_annotations.csv'
validation_points = '/projects/0/ismi2018/FINALPROJECTS/LYMPHOCYTE_DETECTION/validation_annotations.csv' #new
data_dir = 'data'
train_masks_dir = data_dir+'/train_masks'
validation_masks_dir = data_dir+'/validation_masks'
results_dir = 'Results'

num_channels = 3  #changing this breaks the entire thing but just so you know we include it as an option. 

gen_patches = False #Whether or not you want to generate new patches. Keep false to use patches stored on disk
train_patches_dir = data_dir+'/train_patches'
validation_patches_dir = data_dir+'/validation_patches'

#Globals for patches.
global_width = 512
global_height = 512
global_patch_amount = 300

In [ ]:
# read in train csv and show what we've got
train_annotations = pd.read_csv(train_points, index_col=False)
train_annotations.describe()

In [ ]:
train_annotations[' pixel_size'].describe()

**What to gather from this?**

We have 50811 points, the pixel sizes vary from 0.12 to 0.24 but the mean is much closer to 0.24 indicating a skewed distribution. Yet, it's good to keep in mind the varying sizes!

### Some statistics on the training data
So the annotations file has the locations of each annotated cell. It would be good to know how many lymphocytes per ROI we can expect. The below query produces an ordered list per image and ROI of how many annotations we have per combination.

In [ ]:
# get annotations per image, bear in mind this only actually obtains the ROIs that have lymphocytes
# note: this specifically returns a dataframe instead of a series so we can plot it with ease
df = train_annotations.groupby([' roi_id','image_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
df.index.name = 'Combination_id'
df.head(10)

## If the jupyter server runs locally (Brian, Gijs)

In [ ]:
#ttrain_dir = 'train_images'
#ttrain_masks_dir = 'train_masks'
#train_points = 'training_annotations.csv'

# Part 1 - Housekeeping
This section defines the convenience methods needed for processing the images.

* Creating various masks (used as ground truth)
    * Square Masks
    * Elipse Masks
    * Circular Masks
    * Mask creation
* Creating patches
    * Patch generation
* Data augmentation generator

In [ ]:
def split(x):
    return x.split(', ')

### 1.1 Creating Masks

In [ ]:
def createSquareMask(h, w, center, radius):
    mask = np.zeros((h, w), dtype=bool)
    lb = center[0] - radius
    rb = center[0] + radius
    ub = center[1] - radius
    bb = center[1] + radius
    mask[ub:bb, lb:rb] = True
    return mask

def createElipseMask(h, w, center, radius):
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + 2*(Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def createCircularMask(h, w, center, radius):
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask
    

def createMaskForImage(img_dir, image_path, center_csv, mask_dir, mask_fn):
    image_name = image_path.split('.png')[0]
    image_id, roi = image_name.split('_ROI_')
    print('Processing {}'.format(image_name))
    
    image = plt.imread(join(img_dir, image_path))
    mask = np.zeros((image.shape[0], image.shape[1]))
    pixel_size = 0.24309392273426056 # is this the same for each image?
     
    radius = 2.5 / pixel_size #Brian's was 3.5.
    
    pois = []
    with open(center_csv) as file:
        pois = list(map(split, file.readlines()[1:]))
    
    pois = [poi for poi in pois if (poi[0] == image_id and poi[1] == roi)]

    
    print('Found {} lymphocytes in this image.'.format(len(pois))) # How many lymps are annotated here?
    for poi in pois:
        x = int(float(poi[3]))
        y = int(float(poi[4]))
        shape_mask = mask_fn(image.shape[0], image.shape[1], [x, y], radius)
        mask[shape_mask] = 1
        
    mask_image = Image.fromarray(np.uint8(mask*255), 'L')
    mask_image.save(join(mask_dir, image_name + '_mask.png'))
    print('Created a mask for: {}'.format(image_name))

In [ ]:
training_paths = os.listdir(train_dir)
mask_paths = os.listdir(train_masks_dir)
print('Found {} training images for patch generation.'.format(len(training_paths))) # How many images do we actually have?

# Go through the masks, print progress because this is a long job!
for idx, path in enumerate(training_paths):

    mask_name = path.split('.png')[0] + '_mask.png'
    if mask_name not in mask_paths:
        print('Currently at image {}'.format(idx))
        createMaskForImage(train_dir, path, train_points, train_masks_dir, createElipseMask)

### 1.2 Patch Generator
We experimented with multiple ways of generating patches. In this part of the Notebook the different patch generators are implemented.

#### 1.2.1 Split patches
This patch generator splits the images that have a mask into smaller images of given dementions. If the sub-image contains lymphocytes it, and its mask, will be added to the list of patches. This patch generator stops when the patch_amount is reached. if patch_amount <= 0 it will generate patches for all images that have a mask.

Example call: *generate_split_patches(train_dir, train_masks_dir, 0, global_width, global_height, train_points)*

In [ ]:
def split_img_in_patches(img, msk, width, height, pois):
    horizontal_amount = math.floor(img.shape[0]/width)
    vertical_amount   = math.floor(img.shape[1]/height)
    patches = []
    print("getting {}*{}={} patches from image (dimensions:{}x{})".format(horizontal_amount, vertical_amount, horizontal_amount*vertical_amount, img.shape[0], img.shape[1]))
    for x in range(0, horizontal_amount):
        for y in range(0, vertical_amount):
            x1 = x*width
            y1 = y*height
            lymps = [poi for poi in pois if (float(poi[3]) >= x and float(poi[3]) <= x1 and float(poi[4]) >= y and float(poi[4]) <= y1)]
            if(len(lymps) > 0):
                patches.append( (np.array(img[x:x1,y:y1,:]), np.array(msk[x:x1,y:y1])) )
    return patches

In [ ]:
#if patch_amount <= 0 all images that have a mask will be patched!
def generate_split_patches(img_dir, msk_dir, patch_amount, width, height, csv_file):
    patches = []
    all_img_paths = os.listdir(img_dir)
    img_paths = random.sample(all_img_paths, patch_amount)
    with open(csv_file) as file:
        pois = list(map(split, file.readlines()[1:]))
    msk_paths = os.listdir(msk_dir)
    for msk_path in msk_paths:
        img_base_name = msk_path.split("_mask")[0]
        img_id, roi = img_base_name.split('_ROI_')
        _pois = [poi for poi in pois if (poi[0] == img_id and poi[1] == roi)]
        print("patching {}".format(img_base_name+".png"))
        patches.extend(split_img_in_patches(imread(join(img_dir,img_base_name+".png")), imread(join(msk_dir,img_base_name+"_mask.png")), width, height, _pois))
        if(patch_amount > 0 and len(patches)>=patch_amount):
            break
    print("FINISHED PATCHING")
    return patches if (patch_amount < 1) else patches[:patch_amount]
        
#generate_split_patches(train_dir, train_masks_dir, 0, global_width, global_height, train_points)

#### 1.2.2 'Best' patches
This patch generator checks the .csv file for the patch (of given dimensions) on an image that contains the most lymphocytes.
This patch, and its mask, will be added to the list of patches. This patch generator is 'embedded' in the next patch generator and can be called by toggling the last boolean parameter to True.

Example call: *generate_patches(train_dir, train_masks_dir, 300, global_width, global_height, train_points, True)*

In [ ]:
def get_best_patch_coordinate(csv_file, img_id, roi, width, height):
    best_x_coord = 0
    best_y_coord = 0
    with open(csv_file) as file:
        pois = list(map(split, file.readlines()[1:]))
    pois = [poi for poi in pois if (poi[0] == img_id and poi[1] == roi)]
    best_x_coord = get_best_from_column(pois, width, 3)
    pois = [poi for poi in pois if (float(poi[3]) >= best_x_coord and float(poi[3]) <= best_x_coord + width)]
    best_y_coord = get_best_from_column(pois, height, 4)
    print("Best coordinate for {}_ROI_{} is {}X{}".format(img_id, roi, int(best_x_coord), int(best_y_coord)))
    return (best_x_coord, best_y_coord)
    
def get_best_from_column(pois, length, column):
    best_coord = 0
    best_amount = 0
    for i in range(0, len(pois)):
        cur_coord = float(pois[i][column])
        cur_amount = len([poi for poi in pois if (float(poi[column]) >= cur_coord and float(poi[column]) <= cur_coord + length)])
        if(cur_amount > best_amount):
            best_coord = cur_coord
            best_amount = cur_amount
    print("best index on column {} is {}  ({} results)".format(column, best_coord, best_amount))
    return best_coord 

In [ ]:
#TESTING get_best_patch_coordinate function
def test_get_best_patch_coordinate():
    #all_img_paths = os.listdir(train_dir)
    #img_name = random.sample(all_img_paths, 1)[0].split(".png")[0]
    img_name = "31_CD3_ROI_8"
    img_id, roi = img_name.split('_ROI_')
    _img = imread(join(train_dir,img_name+".png"))
    _msk = imread(join(train_masks_dir,img_name+"_mask.png"))
    coord = get_best_patch_coordinate(train_points, img_id, roi, global_width, global_height)
    x = int(coord[0])
    x1 = x + global_width
    y = int(coord[1])
    y1 = y + global_height
    plt.figure(1, figsize=(18, 6))
    plt.subplot(121)
    plt.imshow(_img[x:x1,y:y1,:])
    plt.subplot(122)
    plt.imshow(_msk[x:x1,y:y1])
    
test_get_best_patch_coordinate()

#### 1.2.3 Random patches
This patch generator generates random patches that contain at least 1 lymphocyte. Duplicate patches should not occur.
To use random patch generation toggle the last boolean parameter to False.

Example call: *generate_patches(train_dir, train_masks_dir, 300, global_width, global_height, train_points, False)*

In [ ]:
patch_log = []
coord_tries = []
def get_random_patch_coordinate(csv_file, img_id, roi, width, height):
    #img_name = img_path.split('.png')[0]
    #img_id, roi = img_name.split('_ROI_')
    pois = []
    
    with open(csv_file) as file:
        pois = list(map(split, file.readlines()[1:]))
    pois = [poi for poi in pois if (poi[0] == img_id)]
    while(len(coord_tries) < len(pois)):
        #for poi in pois:
        p = random.randint(0,len(pois)-1)
        while(p in coord_tries):
            p = random.randint(0,len(pois)-1)
        coord_tries.append(p)
        poi = pois[p]
        x = int(float(poi[3]))
        y = int(float(poi[4]))
        if( (x,y) in patch_log):
            return None
        patch_log.append( (x,y) )
        return (round(x - (width/2)), round(y - (height/2)))
    return (0, 0)

In [ ]:
#TESTING get_best_random_coordinate function
def test_get_random_patch_coordinate():
    all_img_paths = os.listdir(train_dir)
    img_name = random.sample(all_img_paths, 1)[0].split(".png")[0]
    img_id, roi = img_name.split('_ROI_')
    _img = imread(join(train_dir,img_name+".png"))
    _msk = imread(join(train_masks_dir,img_name+"_mask.png"))
    coord = get_random_patch_coordinate(train_points, img_id, roi, global_width, global_height)
    print("Random coordinate for {} is {}X{}".format(img_name, int(coord[0]), int(coord[1])))
    x = int(coord[0])
    x1 = x + global_width
    y = int(coord[1])
    y1 = y + global_height
    plt.figure(1, figsize=(18, 6))
    plt.subplot(121)
    plt.imshow(_img[x:x1,y:y1,:])
    plt.subplot(122)
    plt.imshow(_msk[x:x1,y:y1])
    
test_get_random_patch_coordinate()

In [ ]:
#This function generates 1 random patch from patch_amount random images.

def generate_patches(img_dir, msk_dir, patch_amount, width, height, csv_file, best_patches):
    patches = []
    all_img_paths = os.listdir(img_dir)
    img_paths = random.sample(all_img_paths, patch_amount)
    
    for img_path in img_paths:
        img_name = img_path.split('.png')[0]
        msk_path = img_name + '_mask.png'
        img = imread(join(img_dir,img_path))
        msk = imread(join(msk_dir, msk_path)) # changed june 1 to include params
        msk = np.stack((msk,)*3, 2)

        coord = (0,0)
        img_id, roi = img_name.split('_ROI_')
        if(best_patches == True):
            coord = get_best_patch_coordinate(csv_file, img_id, roi, width, height)
        else:
            coord = get_random_patch_coordinate(csv_file, img_id, roi, width, height)
        
        if(coord != None):
            patch = generate_patch(img,msk,width,height, coord)
            patches.append(patch)
            print("{}: {}  image: {}".format(len(patches), patch[0].shape, msk_path))
    while(best_patches == False and len(patches) < patch_amount):
        print("only {} out of {} patches could be generated, recursing now".format(len(patches), patch_amount))
        np.append(patches, generate_patches(img_dir, msk_dir, patch_amount - (len(patches) -1), width, height), axis=0)
    return patches

#This function generates a single random patch from an image.
def generate_patch(img, msk, width, height, coord):
    padHeight = 0 if height < img.shape[1] else height-img.shape[1]
    padWidth  = 0 if width < img.shape[0] else width-img.shape[0]
    if(img.shape[0] < width or img.shape[1] < height):
        print("padding: {}x{}".format(padWidth, padHeight))
    img = np.pad(img, [(0,padWidth), (0, padHeight), (0,0 )], mode='constant')
    msk = np.pad(msk, [(0,padWidth), (0, padHeight), (0,0 )], mode='constant')
    x = int(coord[0])
    x1 = x + width
    y = int(coord[1])
    y1 = y + height
    if x1 > img.shape[0]:
        print("WARNING x out of bounds ({} > {}). Correcting coordinate to fit image.".format(x1,img.shape[0]))
        x1 = img.shape[0]
        x = x1 - width
    if y1 > img.shape[1]:
        print("WARNING y out of bounds ({} > {}). Correcting coordinate to fit image.".format(y1,img.shape[1]))
        y1 = img.shape[1]
        y = y1 - height
    return (np.array(img[x:x1,y:y1,:]), np.array(msk[x:x1,y:y1,:]))

#### 1.2.4 Patch generation
Here we generate the actual patches that are used for training

In [ ]:
#Generate patches!
patches = generate_patches(train_dir, train_masks_dir, global_patch_amount, global_width, global_height, train_points, True)
#patches = generate_patches(train_dir, train_masks_dir, global_patch_amount, global_width, global_height, train_points, False)
mask_patches = []
img_patches = []
for patch in patches:
    img_patches.append(np.array(patch[0]))
    mask_patches.append(np.array(patch[1]))

print("----------------------------")
print("{} image patches generated.".format(len(img_patches)))
print("{} mask patches generated.".format(len(mask_patches)))
print("img:  {}".format(np.array(img_patches[0]).shape))
print("mask: {}".format(np.array(mask_patches[0]).shape))

### 1.3 Data Augmentation
*Not actually used at the moment.*

In [ ]:
# Define a function to read images from disk and convert them to xyc format in a desire output range.
def load_image(input_path, range_min=0, range_max=1):
    
    # Read image data (x, y, c) [0, 255]
    image = Image.open(input_path)
    
    
    half_the_width = image.size[0] / 2
    half_the_height = image.size[1] / 2
    image = image.crop(
        (
            half_the_width - 100,
            half_the_height - 100,
            half_the_width + 100,
            half_the_height + 100
        )
    )
    
    # Convert image to the correct range
    image = np.asarray(image) / 255

    return image

# Define a function to plot a batch or list of image patches in a grid
def plot_image(images, images_per_row=8):
    
    fig, axs = plt.subplots(int(np.ceil(len(images)/images_per_row)), images_per_row)
    
    c = 0
    for ax_row in axs:
        for ax in ax_row:
            if c < len(images):
                ax.imshow(images[c])
            ax.axis('off')            
            c += 1
    plt.show()
    
training_paths = os.listdir(train_dir)
validation_paths = os.listdir(validation_dir)

training_images = [load_image(join(train_dir, path)) for path in training_paths[:50]]
print('Some training examples (shape {shape}):'.format(shape=training_images[0].shape))
plot_image(training_images[12:16], images_per_row=2)

In [ ]:
# define data preparation
datagen = ImageDataGenerator(rotation_range=90,
                                     horizontal_flip=True, 
                                     vertical_flip=True, 
                                     zoom_range=0.3, 
                                     width_shift_range=.3, 
                                     height_shift_range=.3)

fake_labels = np.random.rand(len(training_images))
fake_labels[fake_labels >= .5] = 1
fake_labels[fake_labels < .5] = 0

datagen.fit(training_images)

for X_batch, y_batch in datagen.flow(np.array(training_images), fake_labels, batch_size=9):
    plot_image(X_batch, images_per_row=3)
    break

## 1.4. Storing and getting patches 
Store everything in these objects to disk. TODO

In [ ]:
def storepatchesondisk(patches, dirname):
    #TODO
    print('Patches stored!')

def getpatchesfromdisk(dirname):
    #TODO
    return patches


## 1.5 Calulate Centroids and frame them
Calculate the centroids per blob, filter for size and extent (a measure of how stringy the blob is) and then store it in a dataframe

In [ ]:
def prepareSubmissionData(image_name, images):
    # define dataframe columns
    columns = ['image_id_roi_id', 'pixel_size','x', 'y', 'score']
   
    #create the dataframe used to store coordinates
    results = pd.DataFrame(columns=columns)
    #per image get the list of centroids
    for idx, image in enumerate(images):
        
        labels = skimage.measure.label(image, background=0)
        regions = skimage.measure.regionprops(labels)
        
        #for each region found, add the data to the dataframe
        for region in regions:
            
            # only select blobs that have more than 250 pixels/voxels. Arbitrary number, needs to be optimal
            if region.area >= 250 and region.extent >= 0.6:
                
                temp = pd.DataFrame([[image_name, 0.243094, region.centroid[0], region.centroid[1], 1]], columns=columns)
                results = results.append(temp, ignore_index=True)
 
    return results

# Part 2 - The U-net Zoo
This is where things go deep. The following  networks are defined

* A tiny one that maxes out at 64 (tinynet)
* A "stardard" U-net that maxes out at 128 (smallnet)
* A deeper one that maxes out at 256 (mednet)
* A "stardard" U-net that maxes out at 512 (unet)
* A deeper one that maxes out at 1024 (yugenet)

Additionally this section contains the following:

* A method to save intermediate best networks
* Some sanity checks: side-by-side plot of images and masks
* Generating suitable training images
* Model compilation 
* Model training

In [ ]:
def get_tinynet(img_rows=512, img_cols=512, channels=3):
    inputs = Input(shape=(None, None, channels))
    s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(s)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)

    up3 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2), conv1], axis=3)
    conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(up3)
    conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
 
    conv4 = Conv2D(1, (1, 1), activation='sigmoid')(conv3)
    model = Model(inputs=inputs, outputs=conv4)
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def get_smallnet(img_rows=512, img_cols=512, channels=3):
    inputs = Input(shape=(None, None, channels))
    s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(s)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up4 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv3), conv2], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv4), conv1], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
 
    conv6 = Conv2D(1, (1, 1), activation='sigmoid')(conv5)
    model = Model(inputs=inputs, outputs=conv6)
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def get_mednet(img_rows=512, img_cols=512, channels=3):
    inputs = Input(shape=(None, None, channels))
    s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(s)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4), conv3], axis=3)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv5), conv2], axis=3)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv6), conv1], axis=3)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
 
    conv8 = Conv2D(1, (1, 1), activation='sigmoid')(conv7)
    model = Model(inputs=inputs, outputs=conv8)
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def get_unet(img_rows=global_height, img_cols=global_width, channels=num_channels):
    # A common problem is that Keras might automatically detect "Channels first" or last. The below notation is "Channels last"
    # Which seems to be the default on cartesius. Problem, we only have once channel if we opt to only use the green channel
    #
    # Regarding input layer size - this does not seem to be our problem (Keras docs):
    # When using this layer as the first layer in a model, provide the keyword argument input_shape (tuple of integers,
    # does not include the sample axis), e.g. input_shape=(128, 128, 3) for 128x128 RGB pictures in 
    # data_format="channels_last".
    
    # So flattening also does not force the data into our format (e.g. by using a Flatten() layer)
    
    # Our error seems to be related to https://github.com/keras-team/keras/issues/6351 which includes many miracle fixes
    # yet nothing for us.
    
    inputs = Input(shape=(None, None, channels))
    s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(s)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    model = Model(inputs=inputs, outputs=conv10)
    # we need to punish overfitting on background harshly until our model improves. Dice coef seems to get used often 
    # in literature, and kaggle. Using binary crossentropy for the moment. Before we were using categorical crossentropy
    
    sgd = keras.optimizers.SGD(lr=0.01) # Zanetta used 0.05, ours gets stuck in a local minimum and never improves. Probably related to training samples #
    # however, we are now encountering this: a never changing loss and val_loss. 
    # https://datascience.stackexchange.com/questions/5706/what-is-the-dying-relu-problem-in-neural-networks
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

## 2.2 An Even Deeper U-net
This one goes down to 1024.

In [ ]:
def get_yugenet(img_rows=512, img_cols=512, channels=3):
    # Deeper I go, deeper down
    # Didn't think it could get any
    # blacker
    # The cold bites, the pressure
    # builds
    # I think I no longer matter
    # https://www.youtube.com/watch?v=1-506_jiYkA
    
    inputs = Input(shape=(None, None, channels))
    s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(s)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    #512
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
    
    #1024
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv6)
    
    #512 again
    up7 = concatenate([Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv6), conv5], axis=3)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv7), conv4], axis=3)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv8), conv3], axis=3)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv9)

    up10 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv9), conv2], axis=3)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv10)

    up11 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv10), conv1], axis=3)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv11)
 
    conv12 = Conv2D(1, (1, 1), activation='sigmoid')(conv11)
    model = Model(inputs=inputs, outputs=conv12)
    # we need to punish overfitting on background harshly until our model improves. Dice coef seems to get used often 
    # in literature, and kaggle. Using binary crossentropy for the moment. Before we were using categorical crossentropy
    
    # Zanetta used a lr of 0.05, ours gets stuck in a local minimum and never improves. Probably related to training samples #
    # however, we are now encountering this: a never changing loss and val_loss. 
    # https://datascience.stackexchange.com/questions/5706/what-is-the-dying-relu-problem-in-neural-networks
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

Network parameters, from Francesco's paper:

* LR = 0.05
* Dropouts 0.5 (when?)
* SGD
* CCR (cross-entropy) loss
* F1-score

* RGB threshold on training imgs

Also 10 epochs with batch size 1. x20 resolution and 128x128 in- and output size 

## 2.3 F1-score
f1-score based on what we had in week 7 or 8. Not used at the moment


In [ ]:
def  f1_scoref1_score(y_true, y_pred):
    # Compute precision, recall and obtain several detection thresholds
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    
    # Compute F1-score and remove numerical problems
    f1 =  2 * (precision * recall) / (precision + recall)
    f1 = f1[~np.isnan(f1)]
    
    return f1

## 2.4 Sanity check: plotting a patch and a corresponding mask

In [ ]:
print(np.asarray(img_patches).shape) #should be (num_samples, 512, 512, 3)
print(np.asarray(mask_patches).shape) #should be (num_samples, 512, 512)
ino = 3

plt.figure(1, figsize=(18, 6))
# for the record, matplotlib uses this short-hand notation to show subplots. The 121 here stands for 1 row, 2 columns, first img
# now you know how to use subplots like a pro.
plt.subplot(121)
plt.imshow(img_patches[ino])

plt.subplot(122)
plt.imshow(mask_patches[ino])

## 2.5 compile model and print model architecture. 
Also defines a model_checkpoint callback.

In [ ]:
model = get_smallnet(global_width, global_height, num_channels)
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
model.summary()

## 2.6 Prepare X_train and y_train sets
* X_train contains the training samples (RGB)
* y_train contains the labels as a binary map. Since the masks are generated as a 3-channel image earlier on, we just need to delete the two extra channels.

In [ ]:
X_train = []
y_train = []
for img in img_patches:
    X_train.append(np.asarray(img[:, :, :]))
for msk in mask_patches:
    y_train.append(np.asarray(msk[:, :,:]))

y_train = np.delete(y_train, 2, 3)
y_train = np.delete(y_train, 1, 3)

#sanity checks.. what sizes are we dealing with?
print(len(X_train))
print(np.shape(X_train))
print(np.shape(y_train))

## 2.7 Plot the mask again. 

In [ ]:
#intermediate level sanity checks
plt.figure(1, figsize=(18, 6))
# for the record, matplotlib uses this short-hand notation to show subplots. The 121 here stands for 1 row, 2 columns, first img
# now you know how to use subplots like a pro.
plt.subplot(121)
plt.imshow(img_patches[ino])

plt.subplot(122)
plt.imshow(np.squeeze(y_train[ino]), cmap='gray')

## 2.8 Finally train the model
Train the model. If all goes well...

In [ ]:
# One epoch generally takes 1 minute with batch size 1, 200 samples. Something to keep in mind..
# the issues with the steady loss and val_loss seem to be related to the so-called dying relu problem. Go figure. Another problem to solve!
# model.fit_generator(imgs_train, imgs_mask_train, batch_size=1, nb_epoch=10, verbose=1, shuffle=True, validation_split=0.2, callbacks=[model_checkpoint])
model.fit(np.array(X_train), y_train, batch_size=8, epochs=4, verbose=1, shuffle=True, validation_split=0.2, callbacks=[model_checkpoint])

# Part 3 - Using the Validation set for a sanity check
The below code does the following things:

* Load the best-yet model 

* load the validation images we've been supplied with
* Generate masks for the validation images
    * Creates a validation_masks_dir if there is none yet.
    * Creates validation masks or loads them
    * Creates patches for the validation set
* Show side-by-side plots of the original validation image and the new mask

* Shove one image in our best model and ask predict() what it makes out of it
    * apply a threshold to the image
* Plot the orgininal image, the defined mask, and the prediction side-by-side

## 3.1 Load the best model

In [ ]:
# load our best model
model1 = keras.models.load_model('weights.h5') #saved by our callback earlier

## 3.2 Generate masks for the validation set

In [ ]:
# if it does not exist already, create the validation masks folder
validation_paths = os.listdir(validation_dir)
if os.path.exists(validation_masks_dir):
    pass
else:
    os.system('mkdir -p ' + validation_masks_dir)

# generate the masks if they are not already present on the system.
mask_paths = os.listdir(validation_masks_dir)

print('Found {} validation images.'.format(len(validation_paths))) # How many images do we actually have?

for idx, path in enumerate(validation_paths):
    mask_name = path.split('.png')[0] + '_mask.png'
    if mask_name not in mask_paths:
        print('Currently at image {}'.format(idx))
        createMaskForImage(validation_dir, path, validation_points, validation_masks_dir, createCircularMask)

## 3.3 Generate patches for the validation set

In [ ]:
vpatches = generate_patches(validation_dir, validation_masks_dir, 20, global_width, global_height, validation_points, True)
vmask_patches = []
vimg_patches = []
for patch in vpatches:
    vimg_patches.append(np.array(patch[0]))
    vmask_patches.append(np.array(patch[1]))

In [ ]:
val_images_p = []
for img in vimg_patches:
    val_images_p.append(np.asarray(img[:, :, :]))
    
#sanity checks.. what sizes are we dealing with?
print(len(val_images_p))
print(np.shape(val_images_p))

## 3.4 Check patches and masks from the validation set

In [ ]:
#show validation image and it's mask - Another sanity check
plt.figure(1, figsize=(18, 6))

plt.subplot(121)
print(vimg_patches[0].shape)
plt.imshow(vimg_patches[1])

plt.subplot(122)
plt.imshow(vmask_patches[1])

## 3.5 Make a prediction!

In [ ]:
# Make some predictions
preds_val = model1.predict(np.asarray(val_images_p), verbose=1)

## 3.6 Threshold prediction
May need to find a better way to do this at some point..

In [ ]:
print(np.amin(preds_val)) # prints min
print(np.ptp(preds_val)) # prints range
print(np.amax(preds_val)) # prints max

In [ ]:
# threshold prediction
preds_val = np.squeeze(preds_val) 
preds_val_t = (preds_val > 0.5)*1.0

In [ ]:
print(np.amin(preds_val_t)) # prints min
print(np.ptp(preds_val_t)) # prints range
print(np.amax(preds_val_t)) # prints max

## 3.7 Checking prediction output

In [ ]:
#sanity check for checking image size
print(np.shape(vimg_patches[0]))
print(np.shape(preds_val[0])) 
print(np.shape(preds_val_t[0]))

In [ ]:
# create a subplot of 1 row
plt.figure(1, figsize=(18, 6))
plt.suptitle('Example of a prediction on the validation set', fontsize=16)

idx = 1 

# plot original image
plt.subplot(151)
print(vimg_patches[idx].shape)
plt.imshow(vimg_patches[idx])
plt.title('Input data')

#plot what the mask should have been
plt.subplot(152)
print(vmask_patches[idx].shape)
plt.imshow(vmask_patches[idx])
plt.title('Actual mask')

#plot raw, unthresholded prediction
plt.subplot(153)
print(preds_val[idx].shape)
plt.imshow(preds_val[idx])
plt.title('Un-thresholded prediction')

#plot thresholded prediction
plt.subplot(154)
plt.imshow(preds_val_t[idx], cmap='gray')
plt.title('Thresholded Prediction')

#plot labels using the measure tools
from skimage.measure import label, regionprops

# what this does is label the different blobs based on the connected component principle
labels = label(preds_val_t[idx], background=0)

# it should show even the tiniest specks as different blobs, this is of course undesirable because these are false positives!
plt.subplot(155)
plt.imshow(labels, cmap='spectral')
plt.title('Identification of different clusters')

In [ ]:
#just another validation of what we're seeing
regions = regionprops(labels)

for region in regions:
    print('Number of pixels in blob: '+ format(region.area))
    print('centroid coordinates of blob: '+format(region.centroid))
    print('extent '+format(region.extent))

In [ ]:
#get full images and their titles
test_names = os.listdir(test_dir)
X_test = []
print('found:' + format(len(test_names))) # number of test images

In [ ]:
# Do a quick sample from the test set.
img = imread(join(test_dir,test_names[2]))
print(img.shape)

from sklearn import feature_extraction
img_p = feature_extraction.image.extract_patches_2d(img, (512, 512), max_patches=3)

print(img_p.shape)

In [ ]:
#show the first two pics of the test data
plt.figure(1, figsize=(18, 6))

plt.subplot(121)
plt.title(format(test_names[0]))
plt.imshow(img_p[0])

In [ ]:
test_preds = model1.predict(img_p)

In [ ]:
#thresholding
preds_test = np.squeeze(test_preds) 
preds_test_t = (preds_test > 0.5)*1.0
print(test_preds.shape)
print(preds_test.shape)
print(preds_test_t.shape)

In [ ]:
# create a subplot of 1 row
plt.figure(1, figsize=(18, 6))
plt.suptitle('Example of a prediction on the tes set', fontsize=16)

# plot original image
plt.subplot(141)
plt.imshow(np.squeeze(img_p[0]))
plt.title('Input data')

#plot raw, unthresholded prediction
plt.subplot(142)
plt.imshow(preds_test[0])
plt.title('Un-thresholded prediction')

#plot thresholded prediction
plt.subplot(143)
plt.imshow(preds_test_t[0], cmap='gray')
plt.title('Thresholded Prediction')

# what this does is label the different blobs based on the connected component principle
labels = label(preds_test_t[0], background=0)

# it should show even the tiniest specks as different blobs, this is of course undesirable because these are false positives!
plt.subplot(144)
plt.imshow(labels, cmap='spectral')
plt.title('Identification of different clusters')

In [ ]:
#try getting the image through the prepareSubmissionData function
#preds_test_t = np.expand_dims(preds_test_t, axis=2)
df = prepareSubmissionData(test_names[1], preds_test_t)
df.head()

In [ ]:
#print to see if it looks right
df.to_csv(index=False)

In [ ]:
# make dir for results
result_dir = 'Results'
# Create directory for the results (if not already existing)
if os.path.exists(result_dir):
    pass
else:
    os.system('mkdir -p ' + result_dir)

In [ ]:
df.to_csv("file.csv", encoding="utf-8", index=False)

# Part 4 - Evaluation on the test set

## 4.1 training on the training + validation set
More data is more better. Lets take the validation set as training data for our final model and using a split of 0.1 or 0.2?

Also worth to be considered is training on full images!

## 4.2 Loading test images
TODO

## 4.3 Make predictions on the test images
TODO

## 4.4 Store test prediction images
TODO

## 4.5 Calculate Lymphocyte centroids
TODO

## 4.6 Convert predictions to a csv submission
TODO

# Part 5 - What is outside the flow of work
Counting the number of lymphocytes was the goal of the project although the evaluation is based on the precision of the network. Therefore the following method returns the number of lymphocytes, given a sample image. 

## 5.1 Counting Lymps

In [ ]:
# function to count the number of lymphocytes in a given image
def count_lymps(image):
    num = skimage.measure.label(image,  return_num=True)[1]
    return num

In [ ]:
#lets test this with a mask!
print('number of blobs: '+format(count_lymps(vmask_patches[idx])))

In [ ]:
# and other things that come up

## 5.999 Print the time needed to execute the entire notebook

In [ ]:
print("--- The entire notebook was done in %s seconds ---" % (time.time() - start_time))